# Detector de Spam
Nesse exercício, faremos um detector de spam usando o classificador Naive Bayes

Para isso, usaremos o dataset ["SMS Spam Collection"](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection), que já está salvo no arquivo 'spam.csv'

Primeiramente, execute a célula seguinte para ler o dataset e ver as suas primeiras linhas:

In [2]:
import pandas as pd
import numpy as np

def preprocess(string):
    string = string.lower()
    for c in ['.', ',', '!', '?', '<', '>', '^', '~', ';', ':', '"', '[', ']', '(', ')', '{', '}', '´', '`', '/', '\\', '|']:
        string = string.replace(c, ' ')
    return string

df = pd.read_csv('spam.csv', encoding = 'latin-1')[['v1', 'v2']]
df = df.rename(columns = {'v1':'class', 'v2' : 'contents'})
df['contents'] = df['contents'].map(preprocess)

df.head()

,class,contents
0,ham,go until jurong point crazy available only ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i don't think he goes to usf he lives aro...


Assim, vemos que a tabela possui duas colunas: _class_ e _contents_
 - A coluna _class_ indica se aquela mensagem foi marcada como spam (mensagens legítimas estão marcadas como "ham")
 - A coluna _contents_ possui o conteúdo da mensagem

## Contagem de Palavras
Para o classificador Naive Bayes, nós precisaremos estimar algumas probabilidades contando quantas vezes cada palavra aparece no dataset.

Para isso, primeiramente precisamos saber quais são as palavras únicas e armazená-las em uma lista.

In [3]:
 df.drop_duplicates(subset = ['contents'])['contents']

0       go until jurong point  crazy   available only ...
1                           ok lar    joking wif u oni   
2       free entry in 2 a wkly comp to win fa cup fina...
3       u dun say so early hor    u c already then say   
4       nah i don't think he goes to usf  he lives aro...
                              ...                        
5567    this is the 2nd time we have tried 2 contact u...
5568                will ì_ b going to esplanade fr home 
5569    pity  * was in mood for that  so   any other s...
5570    the guy did some bitching but i acted like i'd...
5571                           rofl  its true to its name
Name: contents, Length: 5167, dtype: object

In [4]:
unique_words = []

# Itera por cada mensagem do dataframe
for message in df['contents']:
    
    
    # Itera por cada palavra na mensagem atual
    for word in message.split():
        
        if word not in unique_words:
            unique_words.append(word)

### Verificação de erros ###
assert len(unique_words) == 9163, 'O vetor unique_words deveria ter exatamente 9163 itens'
assert len(np.unique(unique_words)) == len(unique_words), 'O vetor unique_words não deveria possuir itens repetidos'

Agora, usaremos um dicionário (_dict_) para contarmos quantas vezes cada palavra ocorre no dataset:

In [11]:
# Inicializa o dicionário com 0 ocorrências para cada palavra
word_counts = {word: 0 for word in unique_words}

# Inicializa a contagem total de palavras
total_words = 0



# Itera por cada mensagem do dataframe
for message in df['contents']:
    
    
    # Itera por cada palavra na mensagem atual
    for word in message.split():
        
        
        word_counts[word] += 1
        total_words += 1
        
                
        

### Verificação de erros ###
assert total_words == 87971, 'Existem 87971 ocorrências de palavras no dataset'
assert np.sum(list(word_counts.values())) == total_words, 'O dicionário word_counts está com o número incorreto de ocorrências totais'
assert word_counts['the'] == 1325, 'A palavra "the" ocorre 1325 vezes no dataset'

Também precisaremos de contar quantas vezes cada palavra ocorre **em cada classe**:

In [13]:
# Inicializa o dicionário e a contagem total de palavras entre as mensagens spam
spam_word_counts = {word: 0 for word in unique_words}
spam_total_words = 0

# Inicializa o dicionário e a contagem total de palavras entre as mensagens ham
ham_word_counts = {word: 0 for word in unique_words}
ham_total_words = 0

# Itera por cada mensagem do dataframe
for i in range(len(df)):
    
    # Itera por cada palavra na mensagem atual
    for word in df['contents'][i].split():

        if df['class'][i] == 'spam':
            spam_word_counts[word] += 1
            spam_total_words += 1
        else:
            ham_word_counts[word] += 1
            ham_total_words += 1
            

        
### Verificação de erros ###
assert spam_total_words == 18724, 'Existem 18724 ocorrências de palavras nas mensagens spam'
assert ham_total_words == 69247, 'Existem 69247 ocorrências de palavras nas mensagens ham'
assert np.sum(list(spam_word_counts.values())) == spam_total_words, 'O dicionário spam_word_counts está com o número incorreto de ocorrências totais'
assert np.sum(list(ham_word_counts.values())) == ham_total_words, 'O dicionário ham_word_counts está com o número incorreto de ocorrências totais'
assert np.all([(spam_word_counts[i] + ham_word_counts[i]) == word_counts[i] for i in unique_words]), 'A soma dos valores nos dicionários ham_word_counts e spam_word_counts não está consistente com a contagem total em word_counts'

## Classificador de uma palavra

Com essas contagens, já podemos criar um classificador de spam de uma única palavra, usando o Teorema de Bayes:

$$P(\text{spam} | \text{palavra}) = \frac{P(\text{palavra} | \text{spam}) P(\text{spam})}{P(\text{palavra})}$$

Vamos estimar cada termo no lado direito da equação:

### 1. Prior $P(\text{spam})$

Essa é a probabilidade de uma mensagem ser spam independente do conteúdo, o que corresponde à porcentagem de mensagens total que é spam:

In [25]:
P_spam = (spam_total_words/total_words)
P_ham = (1-P_spam)


### Verificação de erros ###
assert P_spam == 0.21284286867263075, 'Porcentagem incorreta de mensagens do tipo spam'
assert P_ham == 0.7871571313273693, 'Porcentagem incorreta de mensagens do tipo ham'

### 2. Verossimilhança ou _Likelihood_  $P(\text{palavra} | \text{spam})$

Essa é a probabilidade de uma palavra ocorrer **sabendo** a classificação dela. Pra isso, vamos usar as contagens de palavras pra cada classe:

In [27]:
# Função para encontrar P(word | label)
# Argumentos:
# - word: string, uma única palavra do dataset
# - label: string, "spam" ou "ham"

def Word_Likelihood(word, label):

    if label == 'spam':
        return spam_word_counts[word] / spam_total_words
    else:
        return ham_word_counts[word] / ham_total_words
    
    
### Verificação de erros ###
assert np.all(np.abs((pd.Series(unique_words).map(lambda w: Word_Likelihood(w, 'spam')) * spam_total_words - pd.Series(unique_words).map(lambda w: spam_word_counts[w]))) < 1e-8), 'Função retornando valores incorretos quando label = "spam"'
assert np.all(np.abs((pd.Series(unique_words).map(lambda w: Word_Likelihood(w, 'ham')) * ham_total_words - pd.Series(unique_words).map(lambda w: ham_word_counts[w]))) < 1e-8), 'Função retornando valores incorretos quando label = "ham"'

### 3. Marginal $P(\text{palavra})$

Essa é a probabilidade de uma palavra aparecer independente da classificação, o que podemos encontrar usando a ocorrência das palavras:

In [29]:
# Função para encontrar P(word)
# Argumentos:
# - word: string, uma única palavra do dataset

def Word_Marginal(word):

    return word_counts[word] / total_words
    

### Verificação de erros ###
assert np.all(np.abs((pd.Series(unique_words).map(lambda w: Word_Marginal(w)) * total_words - pd.Series(unique_words).map(lambda w: word_counts[w]))) < 1e-8), 'Função retornando valores incorretos'

### Classificando
Agora, podemos usar os três valores calculados para encontrar a probabilidade de uma palavra ser spam:

In [31]:
# Função para encontrar P(spam | word), retorna a probabilidade de uma palavra ser spam
# Argumentos:
# - word: string, uma única palavra do dataset

def word_is_spam(word):
    
    return P_spam * Word_Likelihood(word, 'spam') / Word_Marginal(word)

### Verificação de erros ###
assert np.all(np.abs((pd.Series(unique_words).map(lambda w: word_is_spam(w)) * pd.Series(unique_words).map(lambda w: Word_Marginal(w)) - P_spam * pd.Series(unique_words).map(lambda w: Word_Likelihood(w,'spam')))) < 1e-8), 'Função retornando valores incorretos'

Podemos agora ver a probabilidade de cada palavra ser de uma mensagem spam:

In [32]:
word_is_spam('free')

0.7870036101083033

In [33]:
word_is_spam('friend')

0.23404255319148937

# Classificador Naive Bayes para textos inteiros
Se quisermos classificar textos inteiros em vez de apenas palavras únicas, podemos usar a suposição de independência condicional:

$$P(\text{texto} | \text{spam}) = P(\text{palavra 1} | \text{spam}) \cdot P(\text{palavra 2} | \text{spam}) \ldots $$

Para isso, nós iremos usar a biblioteca Scikit-Learn, que já tem o classificador Naive Bayes implementado.

Primeiramente, precisamos passar os dados de entrada para o formato esperado pela biblioteca. No caso do Naive Bayes, é necessário usar um formato de onde cada coluna representa o número de ocorrências de uma palavra em cada mensagem. Para esse passo de preprocessamento, podemos usar o CountVectorizer, já implementado na biblioteca (se precisar de ajuda, use o [guia do usuário](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction) ou a [documentação](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html?highlight=countvectorizer#sklearn.feature_extraction.text.CountVectorizer).

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['contents'])


### Verificação de erros ###
assert np.sum(2*X - CountVectorizer().fit_transform(df.contents)) == np.sum(X) , "Erro no preprocessamento dos dados"

Também é necessário transformar a coluna de classes em valores numéricos (1 representando spam e 0 representando não spam). Para isso, podemos usar o LabelEncoder, também da Scikit Learn ([documentação](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) e [guia do usuário](https://scikit-learn.org/stable/modules/preprocessing_targets.html#preprocessing-targets))

In [35]:
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(df['class'])

assert np.sum(y - (df['class'] == 'spam')) == 0, "Erro na codificação das labels"

Além disso, é muito importante separarmos um conjunto de teste para poder avaliarmos nosso modelo depois. No nosso caso, criaremos uma divisão com 20% dos dados para teste usando a função train_test_split ([documentação](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)):

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = None, None, None, None

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

assert X.shape[1] == X_train.shape[1] == X_test.shape[1], 'Erro na separação dos conjuntos'
assert X.shape[0] == X_train.shape[0] + X_test.shape[0], 'Erro na separação dos conjuntos'
assert y.shape[0] == y_train.shape[0] + y_test.shape[0], 'Erro na separação dos conjuntos'
assert y_train.shape[0] == X_train.shape[0] == 4457, 'Erro na proporção de separação: apenas 20% dos dados devem ser de teste'

Agora, usaremos a função MultinomialNB ([documentação](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)) para treinar um modelo Naive Bayes no conjunto de teste:

In [37]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(X_train, y_train)

Por fim, podemos calcular a acurácia do modelo nos dados de teste (se tudo ocorreu bem, o valor deve estar acima de 97%):

In [38]:
from sklearn.metrics import accuracy_score
print('Acurácia: %.2lf%%' % (accuracy_score(y_test, model.predict(X_test))*100))

Acurácia: 97.85%


Para testar o modelo em textos, basta usar o mesmo vetorizador na entrada que usamos no preprocessamento, como na função abaixo:

In [39]:
def pred(message):
    message = preprocess(message)
    inputs = vectorizer.transform([message])
    prob = model.predict_proba(inputs)
    print('spam probability: %.2f %%' % (100*prob[0][1],))

In [40]:
message = 'is this message spam?'
pred(message)

spam probability: 36.17 %


In [41]:
message = 'get an iphone for free now'
pred(message)

spam probability: 77.79 %


In [42]:
message = 'I need to have a meeting with you tomorrow'
pred(message)

spam probability: 0.02 %
